In [ ]:
import pandas as pd
import re
test_data = pd.read_csv('./materials/민원22~24년도통합_test_1010_law.csv', encoding='utf-8-sig')
test_data.dropna(subset=['민원내용'], inplace=True)
test_data.reset_index(drop=True, inplace=True)


# preprocess texts
def clean_cut_text(text):
    text = re.sub(r'\([^)]*\)', '', text) 
    text = re.sub(r'\{[^}]*\}', '', text)
    text = re.sub(r'\[[^]]*\]', '', text) 
    text = re.sub(r'\<[^>]*\>', '', text)

    text = re.sub(r'\r\n', ' ', text)
    text = re.sub(r'[\s]+', ' ', text)
    return text.strip()[:1000]

,index,신청번호,신청일자,민원제목,민원요지,민원내용,처리결과,1차추가답변,1차추가답변일,2차추가답변,2차추가답변일,year,추출된 법,추출된 법(re)
0,9777,1AA-2308-0346918,2023-08-09 10:47,용도변경시 구조내력 허용 기준,구조 안전의 확인 등,"""건축물 내진설계기준 KDS 41 17 00"" 에 따르면 ""기존 부분에 대해서는 ...",평소 국토교통 행정에 관심과 애정을 가져주신 것에 감사드립니다.\r\n귀하께서 신청...,NaN,NaN,NaN,NaN,2023,"Law: 건축법 제19조제7항, 건축법 시행령 제32조제2항, 건축구조기준(KDS ...","['건축법 제19조', '건축법 시행령 제32조']"


In [ ]:
import os
import torch

# set available devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1' 

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0", cache_dir='./models')

savename = 'EEVE-10.8b-sumsep-no12-1000'  # set a name for instruction-tuned model
save_dir = f'./{savename}'  # set a direction for instruction-tuned model
tokenizer.save_pretrained(save_dir)

/opt/conda/envs/vllm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


('./EEVE-10.8b-sumsep-no12-1000/tokenizer_config.json',
 './EEVE-10.8b-sumsep-no12-1000/special_tokens_map.json',
 './EEVE-10.8b-sumsep-no12-1000/tokenizer.json')

In [ ]:
from vllm import LLM

# load instruction-tuned model
llm = LLM(model=save_dir, tensor_parallel_size=2)

2024-11-05 02:39:16,592	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 11-05 02:39:22 config.py:905] Defaulting to use mp for distributed inference
INFO 11-05 02:39:22 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='./EEVE-10.8b-sumsep-no12-1000', speculative_config=None, tokenizer='./EEVE-10.8b-sumsep-no12-1000', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=./EEVE-10.8b-sumsep-no12-10

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:00<00:01,  2.71it/s]
Loading safetensors checkpoint shards:  40% Completed | 2/5 [00:01<00:02,  1.49it/s]
Loading safetensors checkpoint shards:  60% Completed | 3/5 [00:02<00:01,  1.23it/s]
Loading safetensors checkpoint shards:  80% Completed | 4/5 [00:03<00:00,  1.17it/s]


(VllmWorkerProcess pid=3064066) 

Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:04<00:00,  1.16it/s]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:04<00:00,  1.24it/s]


INFO 11-05 02:39:38 model_runner.py:1067] Loading model weights took 10.0642 GB


INFO 11-05 02:39:39 model_runner.py:1067] Loading model weights took 10.0642 GB
INFO 11-05 02:39:45 distributed_gpu_executor.py:57] # GPU blocks: 49002, # CPU blocks: 2730
INFO 11-05 02:39:45 distributed_gpu_executor.py:61] Maximum concurrency for 4096 tokens per request: 191.41x
(VllmWorkerProcess pid=3064066) INFO 11-05 02:39:51 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=3064066) INFO 11-05 02:39:51 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-05 02:39:51 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is 

In [ ]:
from tqdm import tqdm

def get_messages(minwon):
    instruction = f"민원: {minwon}\n\n해당 민원을 가장 잘 설명하는 법률 정보를 한국어로 생성하세요."
    messages = [{'role': 'user', 'content': instruction}]
    return messages


# make prompts for test data
prompt_lst = []
for i in tqdm(range(len(test_data))):
    cur_minwon = test_data.loc[i]['민원내용']
    cur_minwon = clean_cut_text(cur_minwon)
    message = get_messages(cur_minwon)
    message = tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)
    prompt_lst.append(message)

100%|██████████| 6056/6056 [00:01<00:00, 5419.71it/s]


In [ ]:
from vllm import LLM, SamplingParams

sampling_params = SamplingParams(max_tokens=1024)

In [11]:
outputs = llm.generate(prompt_lst, sampling_params)

Processed prompts: 100%|██████████| 6056/6056 [15:11<00:00,  6.64it/s, est. speed input: 2078.80 toks/s, output: 6500.62 toks/s] 


In [ ]:
contents = [output.outputs[0].text for output in outputs]  # the final results

In [14]:
test_data['infenrence'] = contents

In [ ]:
# save the inferenced results

test_data.to_csv('./materials/민원22~24년도통합_test_1014_sepinference.csv', encoding='utf-8-sig', index=False)